In [1]:
import bhnerf
from astropy import units
import jax
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

# Runing on 2 GPUs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-6v8uwegk because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2022-12-23 11:36:16.692362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


# Generate synthetic ALMA lightcurves 
---
Polarized lightcurves corresponding to ALMA observation times on April 11, 2017. \
Generate synthetic hot-spot data and the resulting polarized image-plane.

In [2]:
"""
Load ALMA Data and time average over a window
"""
data_path = '../data/LC_EHT2017_V30_I25_K15/AA_SELFC_LO_Apr11.dat'
alma_lc = pd.read_csv(data_path)

t0 = 9. + 20./60.           # UTC
t7 = t0 + 68./60. + 35./60. # UTC

alma_lc_loops = alma_lc.loc[np.bitwise_and(alma_lc['time']>=t0, alma_lc['time']<=t7)]
window_size = 7
alma_lc_means = alma_lc_loops.rolling(window_size).mean().loc[::window_size].dropna()
t_frames = alma_lc_means['time'].values * units.hr

%matplotlib widget
plt.figure(figsize=(5,3))
plt.errorbar(alma_lc.time,  alma_lc['amp'], yerr=alma_lc['sigma'], fmt='.', ecolor='black')
plt.errorbar(alma_lc_means.time,  alma_lc_means['amp'], fmt='.', color='tab:orange')
plt.fill_between([t0, t7], [3.0, 3.0], alpha=0.3, color='gray')
plt.legend()
plt.ylim([1.5, 3])
plt.title('ALMA lightcurve')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Text(0.5, 1.0, 'ALMA lightcurve')

In [3]:
"""
Compute geodesics, azimuthal velocity field, stokes factors
"""
fov_M = 30.0 
spin = 0.2
inclination = np.deg2rad(20.0)

# Compute geodesics (see Tutorial1)
geos = bhnerf.kgeo.image_plane_geos(
    spin, inclination, 
    num_alpha=64, num_beta=64, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2]
)
t_injection = -float(geos.r_o)

# Keplerian prograde velocity field
Omega = np.sign(spin + np.finfo(float).eps) * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
g = bhnerf.kgeo.doppler_factor(geos, umu)

# To match ~hs inensity=0.3, ~hs pol=0.15 (https://www.aanda.org/articles/aa/pdf/2022/09/aa44493-22.pdf)
flux_scale = 0.05
Q_frac = 0.8

# Polarized emission
stokes = ['I', 'Q', 'U']
b_consts = [0.0, -1.0, 0.0]
b = bhnerf.kgeo.magnetic_field(geos, *b_consts) 
J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=0), 0.0)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [4]:
"""
Generate synthetic hot-spot lightcurves matching ALMA observation times.
"""
rot_angle = np.deg2rad(180)
orbit_radius = 10.0
hs_std = 1.0

emission_hs = flux_scale * bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=rot_angle,
    orbit_radius=orbit_radius,
    std=hs_std,
    r_isco=bhnerf.constants.isco_pro(spin),
    fov=(fov_M, 'GM/c^2')
)
image_plane_hs = bhnerf.emission.image_plane_dynamics(emission_hs, geos, Omega, t_frames, t_injection, J)
lc_hs = image_plane_hs.sum(axis=(-1,-2))

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [5]:
%matplotlib widget
movie_list = [xr.DataArray(image_plane_hs[:,i], dims=['t','alpha','beta']) for i in range(image_plane_hs.shape[1])]
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(movie_list, axes, titles=stokes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget
bhnerf.visualization.plot_stokes_lc(lc_hs, stokes, t_frames, add_mean=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:title={'center':'I lightcurve'}>,
       <AxesSubplot:title={'center':'Q lightcurve'}>,
       <AxesSubplot:title={'center':'U lightcurve'}>,
       <AxesSubplot:title={'center':'Q-U loop'}>], dtype=object)

# Recover 3D emission 
---
Recover the unknown 3D emission directly from the polarized lightcurves using bh-NeRF. \
This recovery is an idealized recovery with no systematic noise modeling.

## No background accretion

In [11]:
%matplotlib inline

import warnings
from bhnerf.optimization import LogFn
warnings.simplefilter("ignore")

batchsize = 6
z_width = 4                # maximum disk width [M]
rmax = fov_M / 2           # maximum recovery radius
rmin = float(geos.r.min()) # minimum recovery radius
hparams = {'num_iters': 50000, 'lr_init': 1e-4, 'lr_final': 1e-6, 'seed': 1}

lc_total = image_plane_hs.sum(axis=(-1,-2))
sigma = [1, 0.1, 0.1]

# Logging 
runname = 'bg_noisy/alma_model.no_noise.spin_{:.1f}.inc_{:.1f}.rot{:.1f}.initkey{}.z_width{}'.format(
    spin, np.rad2deg(inclination), np.rad2deg(rot_angle), hparams['seed'], z_width)

writer = bhnerf.optimization.SummaryWriter(logdir='../runs/{}'.format(runname))
writer.add_images('emission/true', bhnerf.utils.intensity_to_nchw(emission_hs), global_step=0)
log_fns = [
    LogFn(lambda opt: writer.add_scalar('log_loss/train', np.log10(np.mean(opt.loss)), global_step=opt.step)), 
    LogFn(lambda opt: writer.recovery_3d(fov_M, emission_true=emission_hs)(opt), log_period=200),
    LogFn(lambda opt: writer.plot_lc_datafit(opt, lc_total, stokes, t_frames, batchsize=20), log_period=200)
]

# Optimization
train_step = bhnerf.optimization.TrainStep.image(t_frames, lc_total, sigma, dtype='lc')
predictor = bhnerf.network.NeRF_Predictor(rmax, rmin, rmax, z_width)
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], J)

optimizer = bhnerf.optimization.Optimizer(hparams, predictor, raytracing_args, checkpoint_dir='../checkpoints/{}'.format(runname))
optimizer.run(batchsize, train_step, raytracing_args, log_fns=log_fns)

iteration:   0%|          | 0/50000 [00:00<?, ?it/s]

In [12]:
emission_estimate = bhnerf.network.sample_3d_grid(predictor.apply, optimizer.state.params, fov=fov_M)
bhnerf.visualization.ipyvolume_3d(emission_estimate, fov=fov_M, level=[0, 0.2, 0.7])

## Stochastic background accretion

In [14]:
"""
Load a 2D Gaussian Random Field (GRF) and expand it to 3D.
pynoisy: https://github.com/aviadlevis/pynoisy
"""
grf = xr.load_dataarray('../data/grf.seed12.nc')
t_interp = float(grf.t[-1]) * (t_frames - t_frames.min()) / (t_frames.max() - t_frames.min())
image_plane_bg = bhnerf.emission.grf_to_image_plane(grf.interp(t=t_interp), geos, Omega, J, alpha=2.0)

# From https://www.aanda.org/articles/aa/pdf/2022/09/aa44493-22.pdf
I_sha, P_sha, V_sha = 2.4, 0.2, -0.025
image_plane_bg = bhnerf.emission.normalize_stokes(image_plane_bg, I_sha, P_sha)
lc_bg = image_plane_bg.sum(axis=(-1,-2))

image_plane = image_plane_hs + image_plane_bg
lc_total = image_plane.sum(axis=(-1,-2))

In [15]:
%matplotlib widget
movie_list = [xr.DataArray(image_plane[:,i], dims=['t','alpha','beta']) for i in range(image_plane.shape[1])]
fig, axes = plt.subplots(1, 3, figsize=(9,3))
bhnerf.visualization.animate_movies_synced(movie_list, axes, titles=['I', 'Q', 'U'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
"""
Optimization: fit Q, U stokes components 
"""
%matplotlib inline

import warnings
from bhnerf.optimization import LogFn
warnings.simplefilter("ignore")

batchsize = 6
z_width = 4                # maximum disk width [M]
rmax = fov_M / 2           # maximum recovery radius
rmin = float(geos.r.min()) # minimum recovery radius
hparams = {'num_iters': 50000, 'lr_init': 1e-4, 'lr_final': 1e-6, 'seed': 1}

stokes = ['Q', 'U']
_J = J[1:]
lc_total = image_plane.sum(axis=(-1,-2))[:, 1:]
sigma = [0.1, 0.1]

# Logging 
runname = 'bg_noisy/alma_model.spin_{:.1f}.inc_{:.1f}.rot{:.1f}.QU_only.seed12.initkey{}.z_width{}'.format(
    spin, np.rad2deg(inclination), np.rad2deg(rot_angle),hparams['seed'], z_width)

writer = bhnerf.optimization.SummaryWriter(logdir='../runs/{}'.format(runname))
writer.add_images('emission/true', bhnerf.utils.intensity_to_nchw(emission_hs), global_step=0)
log_fns = [
    LogFn(lambda opt: writer.add_scalar('log_loss/train', np.log10(np.mean(opt.loss)), global_step=opt.step)), 
    LogFn(lambda opt: writer.recovery_3d(fov_M, emission_true=emission_hs)(opt), log_period=200),
    LogFn(lambda opt: writer.plot_lc_datafit(opt, lc_total, stokes, t_frames, batchsize=20), log_period=200)
]

# Optimization
train_step = bhnerf.optimization.TrainStep.image(t_frames, lc_total, sigma, dtype='lc')
predictor = bhnerf.network.NeRF_Predictor(rmax, rmin, rmax, z_width)
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], _J)

optimizer = bhnerf.optimization.Optimizer(hparams, predictor, raytracing_args, 
                                          save_period=10000,
                                          checkpoint_dir='../checkpoints/{}'.format(runname))
optimizer.run(batchsize, train_step, raytracing_args, log_fns=log_fns)

iteration:   0%|          | 0/50000 [00:00<?, ?it/s]

In [17]:
emission_estimate = bhnerf.network.sample_3d_grid(predictor.apply, optimizer.state.params, fov=fov_M)
bhnerf.visualization.ipyvolume_3d(emission_estimate, fov=fov_M, level=[0, 0.2, 0.7])

# Visualize recovery

In [19]:
def trim_to_gpu_num(array):
    output = array
    if output.shape[0] % jax.device_count():
        output = output[:-1]
    return output

#checkpoint_dir = '../checkpoints/bg_noisy/alma_model.spin_0.2.inc_20.0.rot180.0.QU_only.seed12.initkey2/'

checkpoint_dir = '../checkpoints/tutorial3/recovery.2022-12-23.11:28:41/'
batchsize = 20

t_frames = trim_to_gpu_num(t_frames)
target = trim_to_gpu_num(lc_total)

train_step = bhnerf.optimization.TrainStep.image(t_frames, lc_total, sigma, dtype='lc')
predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
raytracing_args = bhnerf.network.raytracing_args(geos, Omega, t_injection, t_frames[0], _J)
params = predictor.init_params(raytracing_args)
state = predictor.init_state(params, checkpoint_dir=checkpoint_dir)

loss, movie = bhnerf.optimization.total_movie_loss(batchsize, state, train_step, raytracing_args, return_frames=True)
lc_est = movie.sum(axis=(-1,-2))

In [20]:
%matplotlib widget
movie_list = [xr.DataArray(movie[:,i], dims=['t','alpha','beta']) for i in range(movie.shape[1])]
fig, axes = plt.subplots(1, movie.shape[1], figsize=(3*movie.shape[1], 3))
bhnerf.visualization.animate_movies_synced(movie_list, axes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
%matplotlib widget
axes = bhnerf.visualization.plot_stokes_lc(target, ['Q','U'], t_frames, label='True')
bhnerf.visualization.plot_stokes_lc(lc_est, ['Q','U'], t_frames, axes=axes, color='r', fmt='x', label='Estimate')
for ax in axes:
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
emission_estimate = bhnerf.network.sample_3d_grid(predictor.apply, state.params, fov=fov_M)
bhnerf.visualization.ipyvolume_3d(emission_estimate, fov=fov_M, level=[0, 0.2, 0.7])